In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("Sentiment_train.tsv",delimiter='\t',quoting=3)

### 0 - negative,1 - somewhat negative,2 - neutral,3 - somewhat positive,4 - positive

In [3]:
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [4]:
import tensorflow as tf

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
pd.crosstab(df['Sentiment'],1)

col_0,1
Sentiment,
0,7072
1,27273
2,79582
3,32927
4,9206


In [7]:
from sklearn.cross_validation import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df['Phrase'],df['Sentiment'],test_size=0.20,random_state=42)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(124848,)
(31212,)
(124848,)
(31212,)


In [9]:
from tensorflow.python.keras.preprocessing.text import Tokenizer

In [10]:
topnum=5000

In [11]:
t=Tokenizer(num_words=topnum)

In [12]:
t.num_words

5000

In [13]:
t.fit_on_texts(x_train.tolist())

In [14]:
x_train=t.texts_to_sequences(x_train.tolist())

In [15]:
x_test=t.texts_to_sequences(x_test.tolist())

In [17]:
len(x_train[1])

2

In [18]:
max_len=80

In [19]:
from tensorflow.python.keras.preprocessing import sequence


In [20]:
x_train=sequence.pad_sequences(x_train,maxlen=max_len,padding='post')


In [21]:
x_test=sequence.pad_sequences(x_test,maxlen=max_len,padding='post')

In [22]:
x_test

array([[   6,   11, 3798, ...,    0,    0,    0],
       [ 221,   60,    4, ...,    0,    0,    0],
       [   3,  173,  532, ...,    0,    0,    0],
       ...,
       [   1,  750, 4125, ...,    0,    0,    0],
       [2140,  386,    0, ...,    0,    0,    0],
       [   1,  469, 3132, ...,    0,    0,    0]])

In [23]:
print(x_train.shape)

(124848, 80)


In [24]:
print(x_test.shape)

(31212, 80)


In [25]:
embedding_vector_length = 30

In [26]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Embedding,Flatten,Dropout,Dense

In [27]:
model=Sequential()

In [28]:
model.add(Embedding(input_dim=topnum+1,output_dim=embedding_vector_length,input_length=max_len))

In [29]:
model.add(Flatten())

In [30]:
model.add(Dense(300,activation='relu',input_shape=(80,)))

In [31]:
model.add(Dense(200,activation='relu'))

In [32]:
model.add(Dropout(0.3))

In [33]:
model.add(Dense(100,activation='relu'))

In [34]:
model.add(Dense(60,activation='relu'))

In [35]:
model.add(Dense(5,activation="softmax"))

In [36]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [37]:
#pd.crosstab(y_train,1)
from tensorflow.python.keras.utils import to_categorical
y_train = to_categorical(y_train,num_classes=5)

In [38]:
model.fit(x_train,y_train,# validation_data=(x_test,y_test),
          epochs=10,batch_size=10,shuffle=True)

Epoch 1/10
124848/124848 [==============================] - 429s - loss: 1.1578 - acc: 0.5398   
Epoch 2/10
124848/124848 [==============================] - 432s - loss: 1.0128 - acc: 0.5962   
Epoch 3/10
124848/124848 [==============================] - 424s - loss: 0.9137 - acc: 0.6343   
Epoch 4/10
124848/124848 [==============================] - 434s - loss: 0.8729 - acc: 0.6489   
Epoch 5/10
124848/124848 [==============================] - 434s - loss: 0.8420 - acc: 0.6636   - ETA: 0s - loss: 0.8420 - acc:
Epoch 6/10
124848/124848 [==============================] - 451s - loss: 0.8133 - acc: 0.6771   
Epoch 7/10
124848/124848 [==============================] - 450s - loss: 0.7855 - acc: 0.6914   
Epoch 8/10
124848/124848 [==============================] - 442s - loss: 0.7596 - acc: 0.7041   
Epoch 9/10
124848/124848 [==============================] - 443s - loss: 0.7355 - acc: 0.7146   
Epoch 10/10
124848/124848 [==============================] - 448s - loss: 0.7144 - acc: 0.7260  